In [1]:
import tweepy
import pandas as pd
import json
import csv
import re
import os
import requests
from pymongo import MongoClient
#most of these are not necessary :)

In [ ]:
#from tweepy.auth import OAuthHandler
#may not be necessary

In [ ]:
client = MongoClient()
db= client.testing

In [2]:
#removed keys so twitter doesn't ban me from using their API :)
consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

In [3]:
api = tweepy.API(auth)

In [4]:
#class override- apparently this is SOP for tweepy
class Listener(tweepy.StreamListener):
    
    def on_status(self,status):
        is_retweet = False
        #rt flag, start with false as default
        retweet_text= ""
        if hasattr(status, "retweeted_status"):
            is_retweet=True
            try:
                retweet_text = status.retweeted_status.extended_tweet["full_text"]
            except:
                retweet_text = status.retweeted_status.text
        
        #handles 140+ character tweets
        if hasattr(status, "extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text
        
        
        quoted_text =""
        if hasattr(status, "quoted_status"):
            #check if the QT was 140+ char
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet['full_text']
            else:
                quoted_text = status.quoted_status.text
                
        #some minor text-pre-processing to remove newlines and commas incase those become relevant
        remove = [',','\n']
        for i in remove:
            text = text.replace(i," ")
            quoted_text = quoted_text.replace(i," ")
            retweet_text = retweet_text.replace(i," ")
            
#         with open("data.csv","a",encoding='utf-8') as f:
#             f.write("%s,%s,%s,%s,%s,%s\n"%(status.user.screen_name,is_retweet,status.created_at,text,quoted_text,retweet_text))
        db.processTest.insert({'handle':status.user.screen_name,'RT TF':is_retweet,'time':status.created_at,'tweet':text,'qt':quoted_text,'rt':retweet_text,'in_keyword_db':0})
    def on_error(self,status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()
    
    #def on_data(self,data):
        #print(data)

In [5]:
streamListener = Listener()
stream = tweepy.Stream(auth=api.auth, listener=streamListener, tweet_mode='extended')
# with open('data.csv',"w",encoding='utf-8') as f:
#     f.write("handle,is_rt,date,tweet,qt,rt")
tags = ["a", "the", "i", "you", "u"] #= ['python']
stream.filter(languages=["en"], track=tags)

KeyboardInterrupt: 

Keyboard interrupt because I haven't added any stop mechanisms yet :)